#  create a notebook

#### 1. import libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

#### 2.getting data

In [4]:
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
toronto=pd.DataFrame(df[0])
toronto.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### 3. dropping not assigned borough

In [5]:
toronto_df=toronto[toronto.Borough != "Not assigned"].reset_index(drop=True)
toronto_df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


#### 4. grouping the neighbourhood with same borough

In [6]:
toronto_df2 = toronto_df.groupby(["Postal code", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df2

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West
9,M1N,Scarborough,Birch Cliff / Cliffside West


#### 5.replacing not assigned borough

In [7]:
for index, row in toronto_df2.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df2.head()


,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
toronto_df2.shape

(103, 3)

In [9]:

!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [10]:
import pandas as pd
locations = pd.read_csv("Geospatial_Coordinates.csv")
locations.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:

locations.rename(columns={"Postal Code": "Postal code"}, inplace=True)


In [12]:
toronto_df3 = toronto_df2.merge(locations, on="Postal code", how="left")
toronto_df3.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### getting location of toronto

In [15]:
address = 'toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of toronto are 43.6534817, -79.3839347.


### 6. Map of toronto with neighbourhood superimposed on top

In [18]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df3['Latitude'], toronto_df3['Longitude'], toronto_df3['Borough'], toronto_df3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [29]:
toronto_data = toronto_df3[toronto_df3['Borough'] == 'Etobicoke'].reset_index(drop=True)
toronto_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M8V,Etobicoke,New Toronto / Mimico South / Humber Bay Shores,43.605647,-79.501321
1,M8W,Etobicoke,Alderwood / Long Branch,43.602414,-79.543484
2,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North,43.653654,-79.506944
3,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...,43.636258,-79.498509
4,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...,43.628841,-79.520999


In [30]:
address = 'Etobicoke'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Etobicoke are 43.671459150000004, -79.55249206611668.


### create a map of etobicoke superimposed

In [31]:
# create map of Manhattan using latitude and longitude values
map_Etobicoke = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Etobicoke)  
    
map_Etobicoke

In [41]:
CLIENT_ID = 'SSUTEMIRZRUDPXKK4GGPBMLA1PLYN5TX2UZG3XS4ODTHAB4S' # your Foursquare ID
CLIENT_SECRET = 'UJD4CGNIHVB50WXNJYO2S3W3BTXORDR0ADTEIDBFBFHABXNW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SSUTEMIRZRUDPXKK4GGPBMLA1PLYN5TX2UZG3XS4ODTHAB4S
CLIENT_SECRET:UJD4CGNIHVB50WXNJYO2S3W3BTXORDR0ADTEIDBFBFHABXNW


In [44]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Postal code'],toronto_data['Borough'], toronto_data['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [45]:
# converting the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# column names
venues_df.columns = ['Postal code', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(71, 9)


,Postal code,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M8V,Etobicoke,New Toronto / Mimico South / Humber Bay Shores,43.605647,-79.501321,LCBO,43.602281,-79.499302,Liquor Store
1,M8V,Etobicoke,New Toronto / Mimico South / Humber Bay Shores,43.605647,-79.501321,New Toronto Fish & Chips,43.601849,-79.503281,Restaurant
2,M8V,Etobicoke,New Toronto / Mimico South / Humber Bay Shores,43.605647,-79.501321,Domino's Pizza,43.601583,-79.500905,Pizza Place
3,M8V,Etobicoke,New Toronto / Mimico South / Humber Bay Shores,43.605647,-79.501321,Delicia Bakery & Pastry,43.601403,-79.503012,Bakery
4,M8V,Etobicoke,New Toronto / Mimico South / Humber Bay Shores,43.605647,-79.501321,Lucky Dice Restaurant,43.601392,-79.503056,Café


### number of venues returned for each neigbouhood

In [46]:
venues_df.groupby('Neighborhood').count()

,Postal code,Borough,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,,,
Alderwood / Long Branch,9,9,9,9,9,9,9,9
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood,9,9,9,9,9,9,9,9
Kingsview Village / St. Phillips / Martin Grove Gardens / Richview Gardens,4,4,4,4,4,4,4,4
Mimico NW / The Queensway West / South of Bloor / Kingsway Park South West / Royal York South West,14,14,14,14,14,14,14,14
New Toronto / Mimico South / Humber Bay Shores,13,13,13,13,13,13,13,13
Northwest,2,2,2,2,2,2,2,2
Old Mill South / King's Mill Park / Sunnylea / Humber Bay / Mimico NE / The Queensway East / Royal York South East / Kingsway Park South East,1,1,1,1,1,1,1,1
South Steeles / Silverstone / Humbergate / Jamestown / Mount Olive / Beaumond Heights / Thistletown / Albion Gardens,9,9,9,9,9,9,9,9
The Kingsway / Montgomery Road / Old Mill North,3,3,3,3,3,3,3,3


### number of unique data

In [48]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 43 uniques categories.


### ANALYZING EACH NEIGHBOURHOOD

In [49]:
# one hot encoding
etibioke_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
etibioke_onehot['Neighborhood'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [etibioke_onehot.columns[-1]] + list(etibioke_onehot.columns[:-1])
etibioke_onehot = etibioke_onehot[fixed_columns]

etibioke_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Baseball Field,Beer Store,Burger Joint,Burrito Place,Bus Line,Café,Chinese Restaurant,Coffee Shop,Convenience Store,Cosmetics Shop,Discount Store,Drugstore,Fast Food Restaurant,Flower Shop,Fried Chicken Joint,Golf Course,Grocery Store,Gym,Hardware Store,Intersection,Kids Store,Liquor Store,Middle Eastern Restaurant,Mobile Phone Shop,Park,Pet Store,Pharmacy,Pizza Place,Pool,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Seafood Restaurant,Shopping Plaza,Skating Rink,Social Club,Supplement Shop,Wings Joint
0,New Toronto / Mimico South / Humber Bay Shores,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,New Toronto / Mimico South / Humber Bay Shores,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,New Toronto / Mimico South / Humber Bay Shores,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,New Toronto / Mimico South / Humber Bay Shores,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,New Toronto / Mimico South / Humber Bay Shores,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


###  group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [50]:
etibioke_grouped = etibioke_onehot.groupby('Neighborhood').mean().reset_index()
etibioke_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Baseball Field,Beer Store,Burger Joint,Burrito Place,Bus Line,Café,Chinese Restaurant,Coffee Shop,Convenience Store,Cosmetics Shop,Discount Store,Drugstore,Fast Food Restaurant,Flower Shop,Fried Chicken Joint,Golf Course,Grocery Store,Gym,Hardware Store,Intersection,Kids Store,Liquor Store,Middle Eastern Restaurant,Mobile Phone Shop,Park,Pet Store,Pharmacy,Pizza Place,Pool,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Seafood Restaurant,Shopping Plaza,Skating Rink,Social Club,Supplement Shop,Wings Joint
0,Alderwood / Long Branch,0.000000,0.111111,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.111111,0.222222,0.000000,0.111111,0.0,0.000000,0.000000,0.111111,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000
1,Eringate / Bloordale Gardens / Old Burnhamthor...,0.000000,0.000000,0.000000,0.0,0.111111,0.000000,0.000000,0.00,0.111111,0.000000,0.111111,0.111111,0.111111,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.00,0.000000,0.111111,0.000000,0.111111,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000
2,Kingsview Village / St. Phillips / Martin Grov...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.0,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Mimico NW / The Queensway West / South of Bloo...,0.000000,0.000000,0.071429,0.0,0.000000,0.071429,0.071429,0.00,0.000000,0.000000,0.000000,0.071429,0.000000,0.071429,0.0,0.071429,0.000000,0.000000,0.0,0.071429,0.071429,0.071429,0.000000,0.071429,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.071429,0.071429,0.071429
4,New Toronto / Mimico South / Humber Bay Shores,0.076923,0.000000,0.076923,0.0,0.000000,0.000000,0.000000,0.00,0.153846,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.076923,0.076923,0.076923,0.0,0.000000,0.076923,0.000000,0.000000,0.000000,0.076923,0.000000,0.00,0.000000,0.000000,0.076923,0.076923,0.000000,0.000000,0.0,0.076923,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000
5,Northwest,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Old Mill South / King's Mill Park / Sunnylea /...,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,South Steeles / Silverstone / Humbergate / Jam...,0.000000,0.000000,0.000000,0.0,0.111111,0.000000,0.000000,0.00,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.0,0.111111,0.000000,0.111111,0.0,0.222222,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.111111,0.111111,0.000000,0.000000,0.0,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,The Kingsway / Montgomery Road / Old Mill North,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,

### each neighbourhood along with top five common venues

In [51]:
num_top_venues = 5

for hood in etibioke_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = etibioke_grouped[etibioke_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alderwood / Long Branch----
                venue  freq
0         Pizza Place  0.22
1  Athletics & Sports  0.11
2                 Gym  0.11
3        Skating Rink  0.11
4      Sandwich Place  0.11


----Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood----
            venue  freq
0  Cosmetics Shop  0.11
1      Beer Store  0.11
2       Pet Store  0.11
3  Shopping Plaza  0.11
4            Café  0.11


----Kingsview Village / St. Phillips / Martin Grove Gardens / Richview Gardens----
                 venue  freq
0    Mobile Phone Shop  0.25
1             Bus Line  0.25
2       Sandwich Place  0.25
3          Pizza Place  0.25
4  American Restaurant  0.00


----Mimico NW / The Queensway West / South of Bloor / Kingsway Park South West / Royal York South West----
               venue  freq
0     Hardware Store  0.07
1  Convenience Store  0.07
2    Supplement Shop  0.07
3        Social Club  0.07
4     Sandwich Place  0.07


----New Toronto / Mimico South / Humber Bay Shore

In [52]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [53]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = etibioke_grouped['Neighborhood']

for ind in np.arange(etibioke_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(etibioke_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alderwood / Long Branch,Pizza Place,Athletics & Sports,Skating Rink,Gym,Sandwich Place,Pub,Coffee Shop,Pharmacy,Chinese Restaurant,Fast Food Restaurant
1,Eringate / Bloordale Gardens / Old Burnhamthor...,Pet Store,Shopping Plaza,Beer Store,Liquor Store,Cosmetics Shop,Café,Convenience Store,Pizza Place,Coffee Shop,Wings Joint
2,Kingsview Village / St. Phillips / Martin Grov...,Sandwich Place,Mobile Phone Shop,Bus Line,Pizza Place,Wings Joint,Coffee Shop,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Drugstore
3,Mimico NW / The Queensway West / South of Bloo...,Wings Joint,Kids Store,Bakery,Burger Joint,Burrito Place,Convenience Store,Discount Store,Fast Food Restaurant,Grocery Store,Gym
4,New Toronto / Mimico South / Humber Bay Shores,Café,American Restaurant,Restaurant,Gym,Fast Food Restaurant,Liquor Store,Pharmacy,Fried Chicken Joint,Pizza Place,Seafood Restaurant


###  Cluster Neighborhood

In [70]:
# set number of clusters
kclusters = 5

etibioke_grouped_clustering = etibioke_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(etibioke_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 0, 3, 2, 4, 1], dtype=int32)

In [72]:
#add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

etibioke_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
etibioke_merged = etibioke_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

etibioke_merged.head() # check the last columns!#

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M8V,Etobicoke,New Toronto / Mimico South / Humber Bay Shores,43.605647,-79.501321,2.0,Café,American Restaurant,Restaurant,Gym,Fast Food Restaurant,Liquor Store,Pharmacy,Fried Chicken Joint,Pizza Place,Seafood Restaurant
1,M8W,Etobicoke,Alderwood / Long Branch,43.602414,-79.543484,2.0,Pizza Place,Athletics & Sports,Skating Rink,Gym,Sandwich Place,Pub,Coffee Shop,Pharmacy,Chinese Restaurant,Fast Food Restaurant
2,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North,43.653654,-79.506944,4.0,Pool,River,Park,Chinese Restaurant,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store,Cosmetics Shop
3,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...,43.636258,-79.498509,3.0,Baseball Field,Wings Joint,Coffee Shop,Golf Course,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store,Cosmetics Shop
4,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...,43.628841,-79.520999,2.0,Wings Joint,Kids Store,Bakery,Burger Joint,Burrito Place,Convenience Store,Discount Store,Fast Food Restaurant,Grocery Store,Gym


In [75]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(etibioke_merged['Latitude'], etibioke_merged['Longitude'], etibioke_merged['Neighborhood'], etibioke_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

NameError: name 'rainbowint' is not defined